In [2]:
import os
import json
import pandas as pd
import numpy as np
from datetime import datetime

# String Finder

In [12]:
def search_strings_in_json(directory, strings_to_find):
    matches = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.json'):
                filepath = os.path.join(root, file)
                with open(filepath, 'r', encoding='utf-8') as f:
                    try:
                        content = f.read()
                        for string in strings_to_find:
                            if string in content.lower():
                                matches.append((filepath, string))
                    except Exception as e:
                        print(f"Error reading {filepath}: {e}")
    return matches

# strings_to_find = ['2307148732639320', '2090275401330376', '2090275404663709', '100056956201608']
strings_to_find=['8073719999360913']

#company info and company group:61565211886319 --> people_sets_1 & child_groups, 1910535509304367

#7775445322516960
#565159963841935
#1910351919322726
#100040716736805
# directory = '/Users/dan/Documents/local_code/test/data/woolies/workplace/organization'
# directory = '/Users/dan/Documents/local_code/test/data/woolies/workplace/groups'
# directory = 'Workplace Data Company Information/groups'
# directory = 'Workplace Data Company Information/organization'
# directory = 'Workplace Data Company Information/user_profiles'
directory = 'Workplace Data Company Information/groups'
matches = search_strings_in_json(directory, strings_to_find)

count = 0

for match in matches:
    count+=1
    print(f"Found '{match[1]}' in file: {match[0]}")
print(count)

0


# JSON to pkl converter

In [4]:
import os
import json
import pandas as pd
import itertools
import pickle

def read_json_files(root_dir):
    data = {}
    if root_dir.endswith('.json'):
        temp_key_name = root_dir.split('/')[-1].split('.')[0]
        with open(root_dir, 'r', encoding='utf-8') as f:
            try:
                json_data = json.load(f)
                data[temp_key_name] = json_data
            except json.JSONDecodeError as e:
                print(f"Error reading {root_dir}: {e}")
    else:
        for root, dirs, files in os.walk(root_dir):
            for file in files:
                if file.endswith('.json'):
                    filepath = os.path.join(root, file)
                    with open(filepath, 'r', encoding='utf-8') as f:
                        temp_key_name = root_dir + '/' + file
                        try:
                            json_data = json.load(f)
                            data[temp_key_name] = json_data
                        except json.JSONDecodeError as e:
                            print(f"Error reading {filepath}: {e}")
    return data

def extract_data(data):
    records = []
    columns = set()

    def check_exists_append(record, key, value, columns):
        if record.get(key, False):
            if isinstance(record[key], list):
                record[key].append(value)
            else:
                record[key] = [record[key]] + [value]
        else:
            record[key] = value
            columns.add(key)

    def process_data(item, parent_record, parent_labels):
        new_record = 0
        # Copy the parent record to avoid mutation
        record = parent_record.copy()
        labels = parent_labels.copy()

        # Make a new entry if we're at the top level
        for key in ['timestamp', 'media', 'fbid', 'ent_name']:
            if key in item:
                new_record = 1

        if new_record == 1:
            record = {}.copy()

        # Extract basic fields
        for key in ['timestamp', 'media', 'fbid', 'ent_name']:
            if key in item:
                record[key] = item[key]
                columns.add((key, ""))

        # Process 'label_values'
        label_values = item.get('label_values', [])
        if label_values:
            for lv in label_values:
                ent_field_name = lv.get('ent_field_name', '')
                label = lv.get('label', '')
                key_name = [ent_field_name, label]

                if 'value' in lv or 'timestamp_value' in lv:
                    value = lv.get('value') or lv.get('timestamp_value')
                    if labels:
                        check_exists_append(record, tuple(labels), value, columns)

                        # records.append(record.copy())
                    else:
                        check_exists_append(record, tuple(key_name), value, columns)
                elif 'vec' in lv:
                    # Handle 'dict' and 'vec' recursively
                    nested_items = lv.get('vec')
                    if nested_items:
                        for nested_item in nested_items:
                            process_data({'label_values': [nested_item]}, parent_record, key_name )
                elif 'dict' in lv:
                    # print(lv)
                    for entry in lv.get('dict', {}):
                        value = entry.get('value') or entry.get('timestamp_value')
                        temp_label = entry.get('label', '')
                        temp_key_name = (ent_field_name, temp_label)
                        if labels:
                            check_exists_append(record, tuple(labels + [temp_label]), value, columns)
                        else:
                            check_exists_append(record, tuple(key_name + [temp_label]), value, columns)
        else:
            # Handle case for generic JSON file
            for key, value in item.items():
                check_exists_append(record, tuple([key]), value, columns)
        records.append(record.copy())

    # Iterate through a list
    for data_value in data.values():
        if isinstance(data_value, list):
            for item in data_value:
                process_data(item, {}, [])
        else:
            process_data(data_value, {}, [])


    def create_multiindex_df(data):
        # Function to flatten dictionary and preserve both tuple elements
        def flatten_dict(d):
            flattened = {}
            for k, v in d.items():
                if isinstance(k, tuple):
                    # Store both parts of the tuple
                    flattened[k] = v
                else:
                    # For non-tuple keys, create a tuple with same value
                    flattened[(k, k)] = v
            return flattened
        
        # Flatten all dictionaries
        flattened_data = [flatten_dict(d) for d in data]
        
        # Create initial DataFrame
        df = pd.DataFrame(flattened_data)
        
        # Get all unique column tuples
        columns = df.columns.tolist()
        

        # Create MultiIndex columns
        multi_index = pd.MultiIndex.from_tuples(columns)
        
        # Create final DataFrame with MultiIndex
        final_df = pd.DataFrame(df.values, columns=multi_index)
        
        return final_df
    return create_multiindex_df(records)


def convert_to_csv(root_dir):
    assert not root_dir.endswith('.json'), f"ERROR: Path must be a directory not a file {root_dir}"
    root_temp = ""
    folder_loc = -1
    new_name = "default_csv"
    
    for root, dirs, files in os.walk(root_dir):
        # On first iteration we're setting up temp variables
        if not root_temp:
            root_temp = root.split("/")
            folder_loc = len(root_temp) - 1
            new_name = root_temp[folder_loc] + "_pkl_converted"
        
        # For each file in each directory
        for file in files:
            if file.endswith('.json'):
                # Create new output path
                output_root = root.split("/")
                output_root[folder_loc] = new_name
                output_root = "/".join(output_root)
                
                # Process the file
                filepath = os.path.join(root, file)
                temp_data = read_json_files(filepath)
                temp_records = extract_data(temp_data)
                
                # Create output directory if it doesn't exist
                os.makedirs(output_root, exist_ok=True)
                
                # Save each DataFrame in the dictionary
                output_filename = file.replace('.json', '.pkl')
                output_path = os.path.join(output_root, output_filename)
                temp_records.to_pickle(output_path, protocol=pickle.HIGHEST_PROTOCOL)
                print(f"Saved {output_path}")

In [61]:
def extract_company_group(data_dir):
    with open(data_dir, 'r', encoding='utf-8') as f:
        data_dict = json.load(f)
    
    records = []
    record_template = {(key, key): value for key, value in data_dict.items() if key != 'label_values'}
    

    from copy import deepcopy

    for item in data_dict['label_values']:
        # print(item)
        if not "dict" in item:
            record_template[(item['ent_field_name'], item['label'])] = item.get('value', np.nan)

    for item in data_dict['label_values']:
        if "dict" not in item: continue

        record = deepcopy(record_template)
        # print(item)
        record[("title", "title")] = item['title']
        record_copy = deepcopy(record)
        
        entry_list = item['dict']

        for entry in entry_list:
            for s_entry in entry['dict']:
                # print(s_entry)

                record_copy[(s_entry['ent_field_name'], s_entry.get('label', s_entry['ent_field_name']))] = s_entry['value']

            records.append(deepcopy(record_copy))

    column_index = pd.MultiIndex.from_tuples(list(records[0].keys()))
    df = pd.DataFrame.from_records(records, columns=column_index)

    # print(records)

    return df

extract_company_group('Workplace Data Company Information/organization/company_group_2.json')

,media,fbid,ent_name,title,PrimaryEmail,Name,ID,MembershipType
,media,fbid,ent_name,title,Primary email,Name,ID,Membership type
0,[],229949227363012,EntGroup,ConfirmedMembers,antarpreet.singh2@stores.woolworths.co.nz,Antarpreet Singh,61567152175201,Regular member
1,[],229949227363012,EntGroup,ConfirmedMembers,sandeep.singh28@stores.woolworths.co.nz,Sunny Singh,61566646542809,Regular member
2,[],229949227363012,EntGroup,ConfirmedMembers,samuale.henderson@stores.woolworths.co.nz,Sammie Henderson,61566990990994,Regular member
3,[],229949227363012,EntGroup,ConfirmedMembers,maria.demacedosantana@stores.woolworths.co.nz,Maria De Macedo Santana,61566849996557,Regular member
4,[],229949227363012,EntGroup,ConfirmedMembers,jenil.george@stores.woolworths.co.nz,Jenil George,61567086147263,Regular member
...,...,...,...,...,...,...,...,...
15307,[],229949227363012,EntGroup,FormerMembers,harry.voyce@stores.countdown.co.nz,Harry Voyce,100083975481134,Regular member
15308,[],229949227363012,EntGroup,FormerMembers,robert.barton1@stores.countdown.co.nz,Robert Barton,100093460220088,Regular member
15309,[],229949227363012,EntGroup,FormerMembers,suho.jung@stores.countdown.co.nz,Suho Jung,100089230741824,Regular member


# Example use

In [30]:
# Extract and view single file
temp_data = read_json_files('Workplace Data Company Information/organization/company_group_1.json')
temp_records = extract_data(temp_data)
temp_records

,media,fbid,ent_name,UntranslatedNameRaw,Description,WorkPurpose,PrivacyType,URL,FBID,
,media,fbid,ent_name,Name,Description,Is MCG,Group visibility,URL,FBID,
,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
0,[],229949227363012,EntGroup,Countdown,None,False,Closed,https://www.facebook.com/groups/229949227363012/,229949227363012,None


In [28]:
# Extract and view single file
temp_data = read_json_files('Workplace Data Company Information/organization/promoted_post_notice_1.json')
temp_records = extract_data(temp_data)
temp_records

,timestamp,media,fbid,ent_name,ExpirationTime,PostID,AudienceCount,IsBoosted,PromotionType,IsPromotionDeleted,CreatedByID,MarkedAsSeenInKUSurfaceUsersID,ViewedByID,MarkedAsReadUsersID
,timestamp,media,fbid,ent_name,Expiration time,Post ID,Audience count,Is boosted,Promotion type,Is promotion deleted,Created by ID,Marked seen by user ID,Viewed by user ID,Marked read by user ID
0,1724978756,[],2265335223824392,EntWorkPromotedPostCompanyNotice,1725583554,3235370156596868,22968,True,Marked as key update,False,100094913995670,"[100040814411615, 100091235301625, 10001397865...","[100089856112139, 100040618552646, 10001348405...",NaN
1,1724375729,[],2259493764408538,EntWorkPromotedPostCompanyNotice,1724980523,3228352257298658,22938,True,Marked as key update,False,100011662284821,"[100040814411615, 100091235301625, 10001397865...","[100063061770284, 100023940977454, 10002670309...",NaN
2,1724118068,[],2256917171332864,EntWorkPromotedPostCompanyNotice,1724722862,3225336460933571,22955,True,Marked as key update,False,100054215825181,"[100040814411615, 100091235301625, 61558990851...","[100020447681522, 100040618552646, 10001348405...",NaN
3,1723517090,[],2251394921885089,EntWorkPromotedPostCompanyNotice,1724121888,3218336808300203,22996,True,Marked as key update,False,100094913995670,"[100040814411615, 61558990851990, 100013978657...","[100038750782981, 100024024593910, 10004089942...",NaN
4,1722916709,[],2245812622443319,EntWorkPromotedPostCompanyNotice,1723521505,3211508318983052,23001,True,Marked as key update,False,100094913995670,"[100040592814032, 100040814411615, 10001397865...","[100013967952379, 100040837210131, 10008131724...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1603423327,[],1273028313055093,EntWorkPromotedPostCompanyNotice,1603596120,1528784323984775,305,False,Marked as important,False,100011662284821,NaN,"[61563335427027, 100054541762288, 100090899400...","[100014141507632, 100013540178574, 10001356996..."
296,1603414678,[],1272947319729859,EntWorkPromotedPostCompanyNotice,1603673874,1519423171582230,20858,False,Marked as important,False,100011662284821,NaN,"[100040776703895, 100014908690986, 10001457803...","[100025446586759, 100013765809518, 10004346309..."
297,1603171005,[],1270407993317125,EntWorkPromotedPostCompanyNotice,1603775801,2057885601012002,20716,False,Marked as important,False,100011662284821,NaN,"[61556766643580, 100077569069023, 100074691331...","[100027691823643, 100013418594440, 10001297678..."


# Convert whole directory to pkl

In [ ]:
root_dir = 'workplace'
convert_to_csv(root_dir)